# Capstone Project - Finding the locality for setting up a restaurant
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction

BarBeeQ is an Indian chain of restaurants operating in India at present and it is looking forward to launch an outlet in Toronto, Canada since it has a large Indian population of 78,870. The restaurant is to be opened in area which has lower density of Indian restaurants and is closer to the centre of the city. However, due to cost constraints, the restaurant cannot be set up very close to the city centre. The location has to be selected in such a way that it is also close to the areas majorly populated by Indians such as Etobicoke and Brampton. The target audience in this problem would be the owners of the restaurant chain and they would care about it since the solution to the problem will directly have an impact on the cost of establishing a restaurant, its footfall and consequentially its profit.

## Data

The variables crucial for the problem are:
1.	Few or no Indian restaurants in the locality
2.	The locality should be close both to the city centre and to the areas majorly populated by Indians
The data required for the aforementioned problem will be taken from Foursquare API. It will include: 
1.	List of Indian restaurants in Toronto
2.	Location, distance from the city centre, name of the restaurant, address, zip code.
3.	The radius selected is 20 km for finding the restaurants from Toronto city centre

Coordinates of Toronto city centre will be obtained using Google Maps API geocoding.
The clustering of restaurants will be done through which a neighbourhood that satisfies the aforementioned variables will be selected. For example: It is found that Scarborough satisfies the variables. Then I will explore Scarborough to find the Indian restaurants present and using the variables, I will find a locality within the neighbourhood where the restaurant can be set up. Further, within the selected neighbourhood, restaurants will be clustered to find a locality that fits the aforementioned variables with even greater accuracy.


In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0
  - defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0
  - defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4done

# All requested packages already installed.



In [42]:
from IPython.display import Image 
from IPython.core.display import HTML
from IPython.display import display

## Let's get the data from Foursquare 

We are looking for Indian restaurants and will extract the same from Foursquare.

In [4]:
CLIENT_ID = 'W23V2P2JE0UPIFNKK2IA0134R53D00F5FZWEYE4WQGLLTXA3' 
CLIENT_SECRET = '4I0N25VUSALU5MSLU251LN5CT2MP3GE5UUXMWCKJ403SLR5G' 
VERSION = '20200331'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W23V2P2JE0UPIFNKK2IA0134R53D00F5FZWEYE4WQGLLTXA3
CLIENT_SECRET:4I0N25VUSALU5MSLU251LN5CT2MP3GE5UUXMWCKJ403SLR5G


**Let's get the geographical coordinates for Toronto City.**

In [5]:
address = 'Toronto City, Ontario'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [6]:
search_query = 'Indian'
radius = 20000
print(search_query + ' .... OK!')

Indian .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=W23V2P2JE0UPIFNKK2IA0134R53D00F5FZWEYE4WQGLLTXA3&client_secret=4I0N25VUSALU5MSLU251LN5CT2MP3GE5UUXMWCKJ403SLR5G&ll=43.6534817,-79.3839347&v=20200331&query=Indian&radius=20000&limit=1000'

In [91]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e896ff578a484001bc50ac0'},
 'response': {'venues': [{'id': '4aef8854f964a5201cd921e3',
    'name': 'Aroma Fine Indian Restaurant',
    'location': {'address': '287 King St. W',
     'crossStreet': 'at John St.',
     'lat': 43.646462521503445,
     'lng': -79.38964414801342,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.646462521503445,
       'lng': -79.38964414801342}],
     'distance': 906,
     'postalCode': 'M5V 1J5',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['287 King St. W (at John St.)',
      'Toronto ON M5V 1J5',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586065473',
   

In [9]:
venue=results['response']['venues']

In [1]:
dataframe = json_normalize(venue)
dataframe.head()
dataframe.shape()

NameError: name 'json_normalize' is not defined

**Let's clean the dataframe to prepare it for visualisation.**

In [12]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [14]:
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

In [15]:
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,at John St.,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",906,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",NaN,4aef8854f964a5201cd921e3
1,309 Dhaba Indian Excellence,Indian Restaurant,309 King Street West,NaN,43.646394,-79.390418,"[{'label': 'display', 'lat': 43.64639421226189...",946,M5V 1J5,CA,Toronto,ON,Canada,"[309 King Street West, Toronto ON M5V 1J5, Can...",NaN,4ad4c060f964a5206ff720e3
2,Indian Flavour,Indian Restaurant,123 Dundas St W,btw Elizabeth & Bay,43.655649,-79.384119,"[{'label': 'display', 'lat': 43.65564910619165...",241,NaN,CA,Toronto,ON,Canada,"[123 Dundas St W (btw Elizabeth & Bay), Toront...",NaN,4b2a634af964a52020a824e3
3,Indian Biriyani House,Indian Restaurant,181 Dundas St W,W of Chestnut St,43.655120,-79.386645,"[{'label': 'display', 'lat': 43.65511996683289...",284,M5G 1C7,CA,Toronto,ON,Canada,"[181 Dundas St W (W of Chestnut St), Toronto O...",NaN,4afd920ff964a520ad2822e3
4,Indian Biriyani House,Indian Restaurant,120 Adelaide Street West,at Bay Street,43.650050,-79.380662,"[{'label': 'display', 'lat': 43.65005028051833...",464,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor...",NaN,4fea497c121d2480d3579412
5,Touch - Indian Cuisine,Indian Restaurant,1 Adelaide St. E,at Yonge St,43.649869,-79.378218,"[{'label': 'display', 'lat': 43.64986899674456...",611,NaN,CA,Toronto,ON,Canada,"[1 Adelaide St. E (at Yonge St), Toronto ON, C...",NaN,4d137f8c7a8ba1433cb3d169
6,Mami's Indian Cuisine,Food Truck,NaN,NaN,43.656986,-79.385840,"[{'label': 'display', 'lat': 43.65698561897954...",419,M5G 2N2,CA,Toronto,ON,Canada,"[Toronto ON M5G 2N2, Canada]",NaN,596f9dea1fa7632be2eacab9
7,Joe's Indian Restaurant,Indian Restaurant,NaN,NaN,43.658150,-79.381563,"[{'label': 'display', 'lat': 43.65814977325445...",553,NaN,CA,NaN,NaN,Canada,[Canada],NaN,5165c333e4b07a7ad88d8a69
8,Chadani Indian Cuisine,Indian Restaurant,51 Colborne Street,NaN,43.649153,-79.374814,"[{'label': 'display', 'lat': 43.649153, 'lng':...",878,M5E 0B7,CA,Toronto,ON,Canada,"[51 Colborne Street, Toronto ON M5E 0B7, Canada]",NaN,5c7969857dc9e1002c4e3819
9,Ram's Indian kitchen,Indian Restaurant,199 Bay St 6,NaN,43.648026,-79.379819,"[{'label': 'display', 'lat': 43.648025841, 'ln...",691,M5L 1E2,CA,Toronto,ON,Canada,"[199 Bay St 6, Toronto ON M5L 1E2, Canada]",NaN,5b189725d552c7002c0d64c0


In [16]:
df=dataframe_filtered.set_index('categories')

In [21]:
df_rest=df.loc[['Indian Restaurant', 'Fast Food Restaurant', 'Restaurant', 'Food Truck']]
df_rest.reset_index(drop=False)

,categories,name,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Indian Restaurant,Aroma Fine Indian Restaurant,287 King St. W,at John St.,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",906,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",NaN,4aef8854f964a5201cd921e3
1,Indian Restaurant,309 Dhaba Indian Excellence,309 King Street West,NaN,43.646394,-79.390418,"[{'label': 'display', 'lat': 43.64639421226189...",946,M5V 1J5,CA,Toronto,ON,Canada,"[309 King Street West, Toronto ON M5V 1J5, Can...",NaN,4ad4c060f964a5206ff720e3
2,Indian Restaurant,Indian Flavour,123 Dundas St W,btw Elizabeth & Bay,43.655649,-79.384119,"[{'label': 'display', 'lat': 43.65564910619165...",241,NaN,CA,Toronto,ON,Canada,"[123 Dundas St W (btw Elizabeth & Bay), Toront...",NaN,4b2a634af964a52020a824e3
3,Indian Restaurant,Indian Biriyani House,181 Dundas St W,W of Chestnut St,43.655120,-79.386645,"[{'label': 'display', 'lat': 43.65511996683289...",284,M5G 1C7,CA,Toronto,ON,Canada,"[181 Dundas St W (W of Chestnut St), Toronto O...",NaN,4afd920ff964a520ad2822e3
4,Indian Restaurant,Indian Biriyani House,120 Adelaide Street West,at Bay Street,43.650050,-79.380662,"[{'label': 'display', 'lat': 43.65005028051833...",464,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor...",NaN,4fea497c121d2480d3579412
5,Indian Restaurant,Touch - Indian Cuisine,1 Adelaide St. E,at Yonge St,43.649869,-79.378218,"[{'label': 'display', 'lat': 43.64986899674456...",611,NaN,CA,Toronto,ON,Canada,"[1 Adelaide St. E (at Yonge St), Toronto ON, C...",NaN,4d137f8c7a8ba1433cb3d169
6,Indian Restaurant,Joe's Indian Restaurant,NaN,NaN,43.658150,-79.381563,"[{'label': 'display', 'lat': 43.65814977325445...",553,NaN,CA,NaN,NaN,Canada,[Canada],NaN,5165c333e4b07a7ad88d8a69
7,Indian Restaurant,Chadani Indian Cuisine,51 Colborne Street,NaN,43.649153,-79.374814,"[{'label': 'display', 'lat': 43.649153, 'lng':...",878,M5E 0B7,CA,Toronto,ON,Canada,"[51 Colborne Street, Toronto ON M5E 0B7, Canada]",NaN,5c7969857dc9e1002c4e3819
8,Indian Restaurant,Ram's Indian kitchen,199 Bay St 6,NaN,43.648026,-79.379819,"[{'label': 'display', 'lat': 43.648025841, 'ln...",691,M5L 1E2,CA,Toronto,ON,Canada,"[199 Bay St 6, Toronto ON M5L 1E2, Canada]",NaN,5b189725d552c7002c0d64c0
9,Indian Restaurant,Indian Roti House,NaN,NaN,43.664382,-79.380468,"[{'label': 'display', 'lat': 43.66438249736424...",1245,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,547f94e9498e62f695829624


**Let's visualise the data using folium maps.**

In [43]:
toronto_map=folium.Map(location=[latitude, longitude], zoom_start=11)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Toronto',
    color='blue',
    fill=True,
    fill_color='red',
    fill_opacity=0.5).add_to(toronto_map)


In [44]:
for lat, lng in zip(df_rest.lat, df_rest.lng):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup='Indian Restaurant',
        colour='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.5    
        ).add_to(toronto_map)
    
toronto_map

# Methodology

The Indian restaurants around Toronto City, as visualised above, will be clustered using **K-means clustering** to find clusters with lower restaurant density and the one with the lower concentration of restaurants will be selected for exploration purposes.

After clustering the Indian restaurants, we will find a locality towards the West of Toronto city centre as Etobicoke and Brampton lie towards the west of it such that the density of Indian restaurants in that locality is low and it is also within 9 km from the Toronto City centre and Etobicoke. The distance will be calculated by using the **location coordinates** of the respective locations.

After identifying this locality we will explore the restaurants (all cuisines) around it using **Foursquare API** to check if that area has a high concentration of other restaurants. If that is the case then another locality will have to be selected where competition from other restaurants will be low.

We also want that the **restaurants** within **300 m** of the locality be no more than two and **Indian restaurants** within **1 km** of the locality be no more than one. 

## Analysis

**Let's cluster the Indian restaurants using K-means clustering.**

In [ ]:
df_rest.head(1)

In [29]:
restaurant=df_rest[['city','distance','lat','lng','postalCode']]

In [30]:
restaurant=restaurant.reset_index(drop=True)
restaurant.head()

,city,distance,lat,lng,postalCode
0,Toronto,906,43.646463,-79.389644,M5V 1J5
1,Toronto,946,43.646394,-79.390418,M5V 1J5
2,Toronto,241,43.655649,-79.384119,NaN
3,Toronto,284,43.655120,-79.386645,M5G 1C7
4,Toronto,464,43.650050,-79.380662,M5H 3V1


In [31]:
city_dummies=restaurant['city'].str.get_dummies(" ")
city_dummies.head()

,Etobicoke,Toronto
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [32]:
post_dummies=restaurant['postalCode'].str.get_dummies(" ")
post_dummies.head()

,0B7,1B5,1B8,1C7,1E2,1H2,1J5,1K7,1L8,1Z6,...,M5E,M5G,M5H,M5J,M5L,M5R,M5S,M5V,M6G,M8V
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [33]:
rest_group=pd.concat([restaurant, city_dummies], 1)

In [34]:
rest_grouped=pd.concat([rest_group, post_dummies], 1)
rest_grouped.head()

,city,distance,lat,lng,postalCode,Etobicoke,Toronto,0B7,1B5,1B8,...,M5E,M5G,M5H,M5J,M5L,M5R,M5S,M5V,M6G,M8V
0,Toronto,906,43.646463,-79.389644,M5V 1J5,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Toronto,946,43.646394,-79.390418,M5V 1J5,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Toronto,241,43.655649,-79.384119,NaN,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Toronto,284,43.655120,-79.386645,M5G 1C7,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Toronto,464,43.650050,-79.380662,M5H 3V1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [35]:
rest_grouped.drop(['city','postalCode'], axis=1, inplace=True)

In [36]:
rest_grouped.head()

,distance,lat,lng,Etobicoke,Toronto,0B7,1B5,1B8,1C7,1E2,...,M5E,M5G,M5H,M5J,M5L,M5R,M5S,M5V,M6G,M8V
0,906,43.646463,-79.389644,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,946,43.646394,-79.390418,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,241,43.655649,-79.384119,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,284,43.655120,-79.386645,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,464,43.650050,-79.380662,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [37]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [38]:
k=4

kmeans=KMeans(n_clusters=k, random_state=0).fit(rest_grouped)

kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 2, 1,
       0, 1, 1, 2, 1, 1, 1, 0, 0, 1, 3, 1, 3])

In [39]:
rest_grouped.insert(1, 'Cluster labels', kmeans.labels_)

In [40]:
rest_grouped.head()

,distance,Cluster labels,lat,lng,Etobicoke,Toronto,0B7,1B5,1B8,1C7,...,M5E,M5G,M5H,M5J,M5L,M5R,M5S,M5V,M6G,M8V
0,906,3,43.646463,-79.389644,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,946,3,43.646394,-79.390418,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,241,3,43.655649,-79.384119,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,284,3,43.655120,-79.386645,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,464,3,43.650050,-79.380662,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [41]:
restaurant_map=folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lng, label in zip(rest_grouped['lat'], rest_grouped['lng'], rest_grouped['Cluster labels']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup='label',
        color=rainbow[label-1],
        fill=True,
        fill_color=rainbow[label-1],
        fill_opacity=0.7).add_to(restaurant_map)
    
restaurant_map

It can be seen that in the area between **Brampton** and **Downsview Airport**, there is not a single Indian restaurant. Whereas, a lot of Indians live in Brampton and Etobicoke. Therefore, a locality towards the North-West of Toronto City will be suitable.

So, let's explore the area around **Indian Grove** near **Dundas West** to see if satisfies the stakeholder's requirement.

In [65]:
address1 = 'Indian Grove, Toronto, Ontario'

geolocator = Nominatim(user_agent="tpia_explorer")
location = geolocator.geocode(address1)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate are 43.6562633, -79.459343.


In [66]:
search_query = 'restaurants'
radius = 20000
print(search_query + ' .... OK!')

restaurants .... OK!


In [67]:
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url1

'https://api.foursquare.com/v2/venues/search?client_id=W23V2P2JE0UPIFNKK2IA0134R53D00F5FZWEYE4WQGLLTXA3&client_secret=4I0N25VUSALU5MSLU251LN5CT2MP3GE5UUXMWCKJ403SLR5G&ll=43.6534817,-79.3839347&v=20200331&query=restaurants&radius=20000&limit=1000'

In [68]:
results1=requests.get(url1).json()

In [69]:
venue1=results1['response']['venues']

In [70]:
dataframe1 = json_normalize(venue1)
dataframe1.head()

C:\Users\Shivam\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4b295e10f964a520ba9d24e3,Richtree Natural Market Restaurants,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1586064227,False,14 Queen St W,43.652614,-79.380231,"[{'label': 'display', 'lat': 43.65261436174172...",313,M5H 3X4,CA,Toronto,ON,Canada,"[14 Queen St W, Toronto ON M5H 3X4, Canada]",NaN,NaN
1,4d14c68e380a8eec7b2185fa,Imago Restaurants,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1586064227,False,"14 Duncan Street, Suite 203",43.647910,-79.390810,"[{'label': 'display', 'lat': 43.64791, 'lng': ...",831,M5H 3G2,CA,Toronto,ON,Canada,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...",NaN,NaN
2,4d8a7d78bc848cfabd63db2b,Valens Restaurants,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1586064227,False,19 Baldwin Street,43.656096,-79.392839,"[{'label': 'display', 'lat': 43.65609618431269...",773,NaN,CA,Toronto,ON,Canada,"[19 Baldwin Street, Toronto ON, Canada]",NaN,NaN
3,53a07ba3498ee8946e98a7de,Marigold Indian Bistro | Indian Restaurants in...,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1586064227,False,552 Mt Pleasant,43.644302,-79.390002,"[{'label': 'display', 'lat': 43.64430171166487...",1132,M4S 2M6,CA,Toronto,ON,Canada,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]",NaN,NaN
4,55eba3b8498e808c93025230,Restaurants Canada,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1586064227,False,1155 Queen Street West,43.642628,-79.425337,"[{'label': 'display', 'lat': 43.64262834138462...",3547,M6J 1J4,CA,Toronto,ON,Canada,"[1155 Queen Street West, Toronto ON M6J 1J4, C...",NaN,NaN


In [71]:
filtered_columns1 = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns1]

In [72]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [73]:
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

In [74]:
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]

dataframe_filtered1

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,43.652614,-79.380231,"[{'label': 'display', 'lat': 43.65261436174172...",313,M5H 3X4,CA,Toronto,ON,Canada,"[14 Queen St W, Toronto ON M5H 3X4, Canada]",NaN,4b295e10f964a520ba9d24e3
1,Imago Restaurants,Restaurant,"14 Duncan Street, Suite 203",43.647910,-79.390810,"[{'label': 'display', 'lat': 43.64791, 'lng': ...",831,M5H 3G2,CA,Toronto,ON,Canada,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...",NaN,4d14c68e380a8eec7b2185fa
2,Valens Restaurants,Restaurant,19 Baldwin Street,43.656096,-79.392839,"[{'label': 'display', 'lat': 43.65609618431269...",773,NaN,CA,Toronto,ON,Canada,"[19 Baldwin Street, Toronto ON, Canada]",NaN,4d8a7d78bc848cfabd63db2b
3,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,43.644302,-79.390002,"[{'label': 'display', 'lat': 43.64430171166487...",1132,M4S 2M6,CA,Toronto,ON,Canada,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]",NaN,53a07ba3498ee8946e98a7de
4,Restaurants Canada,Office,1155 Queen Street West,43.642628,-79.425337,"[{'label': 'display', 'lat': 43.64262834138462...",3547,M6J 1J4,CA,Toronto,ON,Canada,"[1155 Queen Street West, Toronto ON M6J 1J4, C...",NaN,55eba3b8498e808c93025230
5,Subway,Sandwich Place,"123 Queen Street West, Suite # C72",43.651194,-79.384234,"[{'label': 'display', 'lat': 43.65119366354807...",255,M5H 3M9,CA,Toronto,ON,Canada,"[123 Queen Street West, Suite # C72 (At Sherat...","At Sheraton Toronto Centre, Concourse",50aff967e4b07ba2ce4a1c3c
6,Subway,Sandwich Place,101 College St,43.659875,-79.388606,"[{'label': 'display', 'lat': 43.65987502486328...",805,M5G 1L7,CA,Toronto,ON,Canada,"[101 College St, Toronto ON M5G 1L7, Canada]",NaN,4be9a0576295c9b61e738508
7,Subway,Sandwich Place,"200 Wellington St West, Unit 340",43.645951,-79.387824,"[{'label': 'display', 'lat': 43.64595052717185...",894,M5V 2C7,CA,Toronto,ON,Canada,"[200 Wellington St West, Unit 340, Toronto ON ...",NaN,4b5f325ff964a52093ac29e3
8,Subway,Sandwich Place,287 King St. W,43.646459,-79.389506,"[{'label': 'display', 'lat': 43.64645929083939...",901,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,4c7e90a5d51ea1430508589f
9,Subway,Sandwich Place,"88 Queens Quay West, Unit Fc-8",43.641058,-79.378586,"[{'label': 'display', 'lat': 43.64105847366237...",1448,M5J 0B8,CA,Toronto,ON,Canada,"[88 Queens Quay West, Unit Fc-8, Toronto ON M5...",NaN,59f7478a6bd36b5ffec10c9d


In [97]:
explore_map=folium.Map(location=[latitude1, longitude1], zoom_start=11)

folium.features.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    popup='Indian Grove',
    color='blue',
    fill=True,
    fill_color='red',
    fill_opacity=0.3).add_to(explore_map)

for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        colour='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.5    
        ).add_to(explore_map)
    
explore_map


We see that there are many restaurants around the Indian Grove area.

#### Let's find out which are the 10 nearest restaurants to the Indian Grove

In [76]:
dataframe_filtered1.sort_values(by=['distance'], ascending=True, inplace=True)

In [77]:
nearby_restaurants=dataframe_filtered1.head(10)

In [78]:
nearby_restaurants

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
5,Subway,Sandwich Place,"123 Queen Street West, Suite # C72",43.651194,-79.384234,"[{'label': 'display', 'lat': 43.65119366354807...",255,M5H 3M9,CA,Toronto,ON,Canada,"[123 Queen Street West, Suite # C72 (At Sherat...","At Sheraton Toronto Centre, Concourse",50aff967e4b07ba2ce4a1c3c
0,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,43.652614,-79.380231,"[{'label': 'display', 'lat': 43.65261436174172...",313,M5H 3X4,CA,Toronto,ON,Canada,"[14 Queen St W, Toronto ON M5H 3X4, Canada]",NaN,4b295e10f964a520ba9d24e3
2,Valens Restaurants,Restaurant,19 Baldwin Street,43.656096,-79.392839,"[{'label': 'display', 'lat': 43.65609618431269...",773,NaN,CA,Toronto,ON,Canada,"[19 Baldwin Street, Toronto ON, Canada]",NaN,4d8a7d78bc848cfabd63db2b
6,Subway,Sandwich Place,101 College St,43.659875,-79.388606,"[{'label': 'display', 'lat': 43.65987502486328...",805,M5G 1L7,CA,Toronto,ON,Canada,"[101 College St, Toronto ON M5G 1L7, Canada]",NaN,4be9a0576295c9b61e738508
1,Imago Restaurants,Restaurant,"14 Duncan Street, Suite 203",43.647910,-79.390810,"[{'label': 'display', 'lat': 43.64791, 'lng': ...",831,M5H 3G2,CA,Toronto,ON,Canada,"[14 Duncan Street, Suite 203, Toronto ON M5H 3...",NaN,4d14c68e380a8eec7b2185fa
7,Subway,Sandwich Place,"200 Wellington St West, Unit 340",43.645951,-79.387824,"[{'label': 'display', 'lat': 43.64595052717185...",894,M5V 2C7,CA,Toronto,ON,Canada,"[200 Wellington St West, Unit 340, Toronto ON ...",NaN,4b5f325ff964a52093ac29e3
8,Subway,Sandwich Place,287 King St. W,43.646459,-79.389506,"[{'label': 'display', 'lat': 43.64645929083939...",901,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,4c7e90a5d51ea1430508589f
3,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,43.644302,-79.390002,"[{'label': 'display', 'lat': 43.64430171166487...",1132,M4S 2M6,CA,Toronto,ON,Canada,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]",NaN,53a07ba3498ee8946e98a7de
9,Subway,Sandwich Place,"88 Queens Quay West, Unit Fc-8",43.641058,-79.378586,"[{'label': 'display', 'lat': 43.64105847366237...",1448,M5J 0B8,CA,Toronto,ON,Canada,"[88 Queens Quay West, Unit Fc-8, Toronto ON M5...",NaN,59f7478a6bd36b5ffec10c9d
10,Subway,Sandwich Place,"222 The Esplanade, Unit 1B",43.649594,-79.365779,"[{'label': 'display', 'lat': 43.64959405877831...",1525,M5A 4M8,CA,Toronto,ON,Canada,"[222 The Esplanade, Unit 1B (at Princess), Tor...",at Princess,53850646498e3e40a5878024


It can be seen that there is only one **restaurant** within 300 metres of Indian Grove and no **Indian restaurants** within 1 km of it. Therefore, the conditions mentioned above are satisfied. Now let's see if the other conditions are satisfied.

#### Now, let's calculate the distance between Toronto City and Indian Grove

In [80]:
from math import radians, sin, cos, acos


slat = radians(latitude1)
slon = radians(longitude1)
elat = radians(latitude)
elon = radians(longitude)
dist_toronto_indiangrove = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))

dist_toronto_indiangrove

print('Distance between Toronto City and Indian Grove is: ',dist_toronto_indiangrove)

Distance between Toronto City and Indian Grove is:  6.0745439422191145


We see that the distance between Toronto city centre and Indian Grove is less than 9 km. Therefore, the condition is satisfied.

#### Let's calculate the distance between Brampton and Indian Grove

In [81]:
address2 = 'Brampton, Ontario'

geolocator = Nominatim(user_agent="br_explorer")
location = geolocator.geocode(address2)
latb = location.latitude
lonb = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latb, lonb))

The geograpical coordinate are 43.6858146, -79.7599337.


In [82]:
slat = radians(latb)
slon = radians(lonb)
elat = radians(latitude1)
elon = radians(longitude1)
dist_brampton_indiangrove = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))

print('Distance between Brampton and Indian Grove is: ',dist_brampton_indiangrove)

Distance between Brampton and Indian Grove is:  24.39854973672062


The distance between Brampton is reasonable given that there is no Indian restaurant towards the West of Toronto.

#### Let's calculate the distance between Etobicoke and Indian Grove

In [89]:
address3 = 'East Mall Park, Ontario'

geolocator = Nominatim(user_agent="ee_explorer")
location = geolocator.geocode(address3)
late = location.latitude
lone = location.longitude
print('The geograpical coordinate are {}, {}.'.format(late, lone))

The geograpical coordinate are 43.642328500000005, -79.55727152047683.


In [90]:
slat = radians(late)
slon = radians(lone)
elat = radians(latitude1)
elon = radians(longitude1)
dist_etobicoke_indiangrove = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))

print('Distance between Etobicoke and Indian Grove is: ',dist_etobicoke_indiangrove)

Distance between Etobicoke and Indian Grove is:  8.030079220403028


We see that the distance between Etobicoke and Indian Grove is less than 9 km. Therefore, this condition too is satisfied.

# Results and Discussion

Through the analysis it was seen that there are a great number of restaurants around Toronto city and that their density goes on decreasing as we move farther from the city centre. Highest concentration of restaurants was seen towards the North and South of city centre. The density of restaurants towards the East and the West of city centre was comparatively lower. However, Brampton and Etobicoke lie on the western side and these areas are majorly populated by Indians. Therefore the region towards the West and North West of city centre was selected.

Indian Grove, a locality towards the West of Toronto city centre, was selected initially primarily due to the high Indian presence in the locality. Indian Grove is a street near the Dundas West street and Bloor Street West. The locality is surrounded by various Indian establishments such as the Indian Road Crescent Junior Public School and Indian Trail. The locality also has numerous parks such as High Park, Baird Park and Lithuania Park. Apart from that, there are shopping plazas and malls in the locality such as Crossways Mall and High Park Plaza. Overall, the locality has commericial spaces, low density of restaurants and an Indian population living in it. Therefore, we decided to explore this area to see its restaurant density.

It was seen that there was only one restaurant within 300 metre of the location and no Indian restaurant within 1 km of it. The conditions that had been set earlier were met. After this, the distance between Toronto city centre and the location was found out and it was less than 9 km. Similarly, the distance between Etobicoke and the location was found out using geographical coordinates, which was also less than 9 km. Therefore, all the conditions that were set were satisfied by this locality.

The result of this analysis was that a locality was identified near Toronto where an Indian restaurant could be set up, subject to the given conditions. This, however, does not imply that it is the most optimal location. There can be more locations that are more optimal than the one suggested by this analysis.Recommended locality should therefore be considered only as a starting point for more detailed analysis which could eventually result in a location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

# Conclusion

Purpose of this project was to identify Toronto areas close to center with low number of restaurants (particularly Indian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Indian restaurant. By clustering the Indian restaurants, a basic idea was obtained about the density of Indian restaurants around Toronto city centre and then find a locality which satisfies some basic requirements regarding existing nearby restaurants and distances from Toronto city centre and Etobicoke. After clustering the Indian restaurants, a location towards the West of Toronto had to be identified. Various conditions mentioned above were taken into consideration and it was ensured that they satisfied and thus, a locality was suggested for setting up the restaurant.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and the recommended locality, taking into consideration additional factors like attractiveness of the location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.